In [43]:
import kagglehub
import os
import pandas as pd
import polars as pl
import statsmodels.api as sm

In [ ]:
#uncomment to download data
#kagglehub.login()
#kagglehub.competition_download('store-sales-time-series-forecasting')


Kaggle credentials set.
Kaggle credentials successfully validated.


In [18]:
os.listdir('..\data')

['holidays_events.csv',
 'oil.csv',
 'sample_submission.csv',
 'stores.csv',
 'test.csv',
 'train.csv',
 'transactions.csv']

In [44]:
train_data = pl.read_csv(r'..\data\train.csv')


In [47]:
# Convert date column
train_data = train_data.with_columns(
    pl.col('date').str.strptime(pl.Date, format='%Y-%m-%d')
)

# Extract year, month, weekday
train_data = train_data.with_columns([
    pl.col('date').dt.year().alias('year'),
    pl.col('date').dt.month().alias('month'),
    pl.col('date').dt.weekday().alias('weekday_num')
])

train_data

id,date,store_nbr,family,sales,onpromotion,year,month,weekday_num
i64,date,i64,str,f64,i64,i32,i8,i8
0,2013-01-01,1,"""AUTOMOTIVE""",0.0,0,2013,1,2
1,2013-01-01,1,"""BABY CARE""",0.0,0,2013,1,2
2,2013-01-01,1,"""BEAUTY""",0.0,0,2013,1,2
3,2013-01-01,1,"""BEVERAGES""",0.0,0,2013,1,2
4,2013-01-01,1,"""BOOKS""",0.0,0,2013,1,2
…,…,…,…,…,…,…,…,…
3000883,2017-08-15,9,"""POULTRY""",438.133,0,2017,8,2
3000884,2017-08-15,9,"""PREPARED FOODS""",154.553,1,2017,8,2
3000885,2017-08-15,9,"""PRODUCE""",2419.729,148,2017,8,2


In [69]:
train_data.describe()

statistic,id,date,store_nbr,family,sales,onpromotion,year,month,weekday_num
str,f64,str,f64,str,f64,f64,f64,f64,f64
"""count""",3.000888e6,"""3000888""",3.000888e6,"""3000888""",3.000888e6,3.000888e6,3.000888e6,3.000888e6,3.000888e6
"""null_count""",0.0,"""0""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",1500443.5,"""2015-04-24 08:27:04.703000""",27.5,null,357.775749,2.60277,2014.837886,6.207838,3.997031
"""std""",866281.891642,null,15.585787,null,1101.997721,12.218882,1.345518,3.385668,2.00074
"""min""",0.0,"""2013-01-01""",1.0,"""AUTOMOTIVE""",0.0,0.0,2013.0,1.0,1.0
"""25%""",750222.0,"""2014-02-27""",14.0,null,0.0,0.0,2014.0,3.0,2.0
"""50%""",1.500444e6,"""2015-04-25""",28.0,null,11.0,0.0,2015.0,6.0,4.0
"""75%""",2.250665e6,"""2016-06-19""",41.0,null,195.847,0.0,2016.0,9.0,6.0
"""max""",3.000887e6,"""2017-08-15""",54.0,"""SEAFOOD""",124717.0,741.0,2017.0,12.0,7.0


In [68]:
train_data.glimpse()

Rows: 3000888
Columns: 9
$ id           <i64> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
$ date        <date> 2013-01-01, 2013-01-01, 2013-01-01, 2013-01-01, 2013-01-01, 2013-01-01, 2013-01-01, 2013-01-01, 2013-01-01, 2013-01-01
$ store_nbr    <i64> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ family       <str> 'AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI'
$ sales        <f64> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
$ onpromotion  <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ year         <i32> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013
$ month         <i8> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ weekday_num   <i8> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2



In [64]:
summary = (
    train_data.group_by('year', 'month')
      .agg(pl.col('sales').sum())
      .pivot(
          index='year',
          values='sales',
          columns='month'
      )
      .with_columns(
          [pl.col(f'{col}').map_elements(lambda x: f'${x:,.0f}', return_dtype=pl.Utf8) for col in range(1, 13)]
      )
)

C:\Users\Leo\AppData\Local\Temp\ipykernel_11292\3758019131.py:2: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  train_data.group_by('year', 'month')


In [65]:
summary

year,7,2,3,10,6,12,11,8,9,1,5,4
i32,str,str,str,str,str,str,str,str,str,str,str,str
2016,"""$23,462,672""","""$21,947,409""","""$23,131,781""","""$24,030,390""","""$22,209,219""","""$29,640,288""","""$24,642,640""","""$22,452,414""","""$22,417,448""","""$23,977,805""","""$24,779,432""","""$25,963,025"""
2013,"""$11,257,401""","""$9,658,960""","""$11,428,497""","""$11,775,620""","""$11,689,344""","""$15,803,117""","""$12,356,559""","""$11,737,789""","""$11,792,933""","""$10,327,625""","""$11,597,704""","""$10,993,465"""
2015,"""$22,209,619""","""$13,742,396""","""$15,598,608""","""$23,878,268""","""$21,615,360""","""$27,243,982""","""$22,804,953""","""$22,963,674""","""$23,240,882""","""$14,896,922""","""$17,730,368""","""$14,955,068"""
2014,"""$19,421,891""","""$12,038,353""","""$20,365,584""","""$20,396,101""","""$13,319,958""","""$24,340,454""","""$20,531,635""","""$13,885,176""","""$20,022,416""","""$18,911,641""","""$13,379,785""","""$12,861,251"""
2017,"""$27,011,478""","""$23,250,112""","""$26,704,018""",null,"""$25,682,822""",null,null,"""$12,433,323""",null,"""$26,328,160""","""$26,911,847""","""$25,895,308"""


In [72]:

def calculate_missing_percentage(df):
    """
    Calculate the percentage of missing values for each column in a Polars DataFrame.
    
    Parameters:
    -----------
    df : pl.DataFrame
        Input Polars DataFrame
    
    Returns:
    --------
    pl.DataFrame
        DataFrame with columns: 
        - 'column_name': name of the column
        - 'total_rows': total number of rows
        - 'missing_count': number of missing values
        - 'missing_percentage': percentage of missing values
    """
    # Get total number of rows
    total_rows = df.height
    
    # Calculate missing values
    missing_stats = (
        df.null_count()
        .melt(variable_name="column_name", value_name="missing_count")
        .with_columns([
            pl.lit(total_rows).alias("total_rows"),
            ((pl.col("missing_count") / total_rows * 100).round(2)).alias("missing_percentage")
        ])
        .select([
            "column_name", 
            "total_rows", 
            "missing_count", 
            "missing_percentage"
        ])
    )
    
    return missing_stats

calculate_missing_percentage(train_data)

C:\Users\Leo\AppData\Local\Temp\ipykernel_11292\3293749318.py:24: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df.null_count()


column_name,total_rows,missing_count,missing_percentage
str,i32,u32,f64
"""id""",3000888,0,0.0
"""date""",3000888,0,0.0
"""store_nbr""",3000888,0,0.0
"""family""",3000888,0,0.0
"""sales""",3000888,0,0.0
"""onpromotion""",3000888,0,0.0
"""year""",3000888,0,0.0
"""month""",3000888,0,0.0
"""weekday_num""",3000888,0,0.0


In [74]:
null_rows = train_data.filter(pl.col('sales').is_null())
null_rows

id,date,store_nbr,family,sales,onpromotion,year,month,weekday_num
i64,date,i64,str,f64,i64,i32,i8,i8


In [75]:
transactions = pl.read_csv(r'..\data\transactions.csv')

In [76]:
transactions

date,store_nbr,transactions
str,i64,i64
"""2013-01-01""",25,770
"""2013-01-02""",1,2111
"""2013-01-02""",2,2358
"""2013-01-02""",3,3487
"""2013-01-02""",4,1922
…,…,…
"""2017-08-15""",50,2804
"""2017-08-15""",51,1573
"""2017-08-15""",52,2255
